In [1]:
from ttrtypes import BType,Pred,PType,Possibility,Ty,RecType,SingletonType
from records import Rec
from utils import show

# Possibilities
The possibilities facility has been improved.  Judging an object to be of a type (as in `T.judge('c')`) will mean that the object shows up as being of that type in all possibilities.  The possibility `M` will be updated at the point at which you call `T.in_poss(M)`.  If you judge an object to be of a type in a certain possibility (as in `T.in_poss(M).judge('a')` then `a` will show up as being of that type only in possibility `M`.

In [2]:
M = Possibility('M')
M1 = Possibility('M1')
T = BType('T')
T.judge('c')
T.in_poss(M).judge('a')
T.in_poss(M1).judge('b')
print(show(M))
print(show(M1))


M:
_____________________________________________
T: [c, a]
_____________________________________________


M1:
_____________________________________________
T: [c, b]
_____________________________________________



# Learning witness conditions
If you add a witness condition to a type (as in `T1.learn_witness_condition(lambda x: len(x)==2)`) that witness condition will apply in all possibilities.  The witness condition must be a function which returns a boolean.

In [3]:
T1 = BType('T1')
T1.learn_witness_condition(lambda x: len(x)==2)
print(T1.query('aa'))
print(T1.in_poss(M).query('aa'))

True
True


# Learning witness types
You can also add a witness type to a type (as in `T.learn_witness_type(T1)`).  This means that anything of type `T1` (in the current possibility) will also be of type `T` (in that possibility).

In [4]:
T.learn_witness_type(T1)
print(T.query('aa'))
print(T.in_poss(M).query('aa'))
print(T.in_poss(M1).query('bb'))

True
True
True


# Learning witness types for ptypes
Witness types can be added to ptypes.  For example any situation where Sam is whistling may be one where Sam is happy.  So any situation where Sam is whistling is also one where he is happy.  This might be true for Sam but not other people.

In [5]:
Ind = BType('Ind')
Ind.judge('sam')
Ind.judge('kim')
happy = Pred('happy',[Ind])
whistle = Pred('whistle',[Ind])
happySam = PType(happy,['sam'])
whistleSam = PType(whistle,['sam'])
whistleSam.in_poss(M).judge('e')
whistleSam.in_poss(M1).judge('e1')
happySam.learn_witness_type(whistleSam)
print(happySam.in_poss(M).query('e'))
print(happySam.in_poss(M1).query('e1'))
print(show(M))
print(show(M1))

True
True

M:
_____________________________________________
T: [c, a, aa]
T1: [aa]
whistle(sam): [e]
happy(sam): [e]
_____________________________________________


M1:
_____________________________________________
T: [c, b, aa, bb]
T1: [aa, bb]
whistle(sam): [e1]
happy(sam): [e1]
_____________________________________________



# Learning witness functions for predicates
Witness functions may be associated with predicates (as in `happy.learn_witness_fun(lambda args: PType(smile,args))`).  Any ptype constructed with the predicate `happy` will be witnessed by a situation where the argument of the predicate smiles, whoever the argument is.  That is, the predicate brings the witness condition for the ptype along with it.  Witness functions associated with a predicate must be functions which will take the argument list and return a type.  Anything of this type will be a witness for the ptype.

In [6]:
M2 = Possibility('M2')
smile = Pred('smile',[Ind])
smileKim = PType(smile,['kim'])
happyKim = PType(happy,['kim'])
smileKim.in_poss(M2).judge('e2')
happy.learn_witness_fun(lambda args: PType(smile,args))
happyKim.in_poss(M2).query('e2')

True

# Calling external witness conditions in a ptype
You can associate an external witness condition such as one based on a classifier with a predicate by first associating the witness condition with a type and then having the witness function for the predicate return that type, or some type based on that type.

In [7]:
Triple = BType('Triple')
Triple.learn_witness_condition(lambda x: len(x)==3)
Bot = BType('Bot')
Bot.learn_witness_condition(lambda x: not x==x)
threesome = Pred('threesome',[Ind])
threesome.learn_witness_fun(lambda args: RecType({'x':SingletonType(Ind,args[0]),
                                                  'type':SingletonType(Ty,Triple)}) 
                            if Triple.query(args[0]) else Bot) 
                                                
Ind.judge('aaa')
Ind.judge('aa')
r = Rec({'x':'aaa',
        'type':Triple})
r_aa = Rec({'x':'aa',
            'type':Triple})
three_aaa = PType(threesome, ['aaa'])
print(three_aaa.query(r))
three_aa = PType(threesome,['aa'])
print(three_aa.query(r_aa))

True
False
